In [1]:
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import DesiredCapabilities
from bs4 import BeautifulSoup
import pandas as pd
# import datetime as dt
from json import dumps

In [7]:
#Constants
URL_DOMAIN = 'https://teacode.com/online/vak/'
URL_FIRST_PAGE = ''
CSV_DIR = 'csv/'
CSV_FNAME = 'categories-'

# Function to ask yes/no
def q_yn(q, default="yes"):
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(q + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")

# Function to wait Web Driver on specific page
# returns 0 if OK and 1 if there was an error

def wait_pageload(w):
    try:
        WebDriverWait(w, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, 'OntoBox.org')))
    except:
        return 1
    return 0

def parse_diss_text(dtext):
    return dtext

def get_clist(w, url=URL_DOMAIN):
    w.get(url)
    wait_pageload(w)
    print('Page %s is loaded' % url)

    bsObj = BeautifulSoup(w.page_source, "lxml")

    ptable = bsObj.html.body.table.table

    if 'Паспорт специальности' in bsObj.title.string:
        a = ptable.tbody.find_all("tr")[1].td.text
        a = parse_diss_text(a)
        dh = bsObj.html.body.table.tbody.tr.td
        ztable = [[dh.h1.text,dh.h3.text,url,a]]
    else:
        ztable = [([a.text for a in row.find_all("td")] +
                [URL_DOMAIN + j['href'] for j in row.find_all("a", href=True)]+[''])
                for row in ptable.find('tbody').find_all("tr", recursive=False)][1:-1]

    return ztable


def recurse_urls(w):

    def gc(c, n=2):
        return [x[n] for x in c]

    p = get_clist(w)
    
    for i in gc(p):
        j = get_clist(w,i)
        p += j
        if gc(j,3)[0]=='':
            print(p)
            for ii in gc(j):
                jj = get_clist(w,ii)
                p += jj
                if gc(jj,3)[0]=='':
                    print(p)
                    for iii in gc(jj):
                        jjj = get_clist(w, iii)
                        p += jjj
                        if gc(jjj,3)[0]=='':
                            print(p)
                            for iiii in gc(jjj):
                                print(iiii)

    columns = ['code', 'name', 'url', 'text']

    df = pd.DataFrame(p,columns=columns)

    return df

In [3]:
# Set HTTP Headers following the Ethical Web Scraping Practice
# Note: site administrators stay healthy when our bot looks like a human

# Code snippet from @andriy-ivaneyko on StackOverflow
desired_capabilities = DesiredCapabilities.CHROME.copy()

desired_capabilities['chrome.page.customHeaders.Accept'] = '*/*'
desired_capabilities['chrome.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
desired_capabilities['chrome.page.customHeaders.Accept-Language'] = 'ru-RU,en;q=0.8'
desired_capabilities['chrome.page.customHeaders.Cache-Control'] = 'max-age=0'
desired_capabilities['chrome.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/79.0.3945.79 Chrome/79.0.3945.79 Safari/537.36'

w = webdriver.Chrome(desired_capabilities=desired_capabilities)

In [8]:
p = recurse_urls(w)
p

ение прав человека на жизнь, свободу, защиту чести и достоинства. Гарантии политических прав на свободу совести, мнения в печати, на радио, телевидении, в интернете как возможность обеспечения ненасильственных практик.Власть как средство регулирования конфликтов и обеспечения интеграции в обществе. Авторитет власти для управления конфликтными ситуациями. Роль авторитета власти, основанная на заинтересованности сторон, на традиции к повиновению, на статусном авторитете в достижении консенсусных решений и обеспечении согласия в обществе. Этнократические способы властвования, этнические приоритеты в формировании управленческих кадров, механизмов управления, мобилизации и использования ресурсов. Роль международного сообщества и внешних факторов в регулировании конфликтов. Изучение возможностей \nпридания конфликтам цивилизованных форм. \nОтрасль наук: \n\nполитические науки\n'], ['24.00.01', 'Теория и история культуры', 'https://teacode.com/online/vak/p24-00-01.html', ''], ['24.00.03', 'Му

,code,name,url,text
0,01.00.00,Физико-математические науки,https://teacode.com/online/vak/p01-00-00.html,
1,02.00.00,Химические науки,https://teacode.com/online/vak/p02-00-00.html,
2,03.00.00,Биологические науки,https://teacode.com/online/vak/p03-00-00.html,
3,05.00.00,Технические науки,https://teacode.com/online/vak/p05-00-00.html,
4,06.00.00,Сельскохозяйственные науки,https://teacode.com/online/vak/p06-00-00.html,
5,07.00.00,Исторические науки и археология,https://teacode.com/online/vak/p07-00-00.html,
6,08.00.00,Экономические науки,https://teacode.com/online/vak/p08-00-00.html,
7,09.00.00,Философские науки,https://teacode.com/online/vak/p09-00-00.html,
8,10.00.00,Филологические науки,https://teacode.com/online/vak/p10-00-00.html,
9,12.00.00,Юридические науки,https://teacode.com/online/vak/p12-00-00.html,


In [10]:
########
# Main code

csv_name = CSV_FNAME + '-latest.csv'
json_name = CSV_FNAME + '-latest.json'

p.to_csv(csv_name, header=True)
p.to_json(json_name, force_ascii=False, orient='records')

w.quit()